In [2]:
import os
import json
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [31]:
# data loader

# get farm data
def get_farm_data(farm_id):
    # dir = os.path.dirname(__file__)
    dir = os.getcwd()
    farm_data_path = os.path.join(dir, '../data/test/litefarm_test.csv')
    df = pd.read_csv(farm_data_path)
    farm = df.query(f"farm_id == '{farm_id}'")
    area = farm["area_in_m2"].iloc[0] * 0.0001  # Litefarm database use m^2 while holos calculation use ha
    crop_yield = farm["yield_kg"].iloc[0] / area  # kg/ha
    location = [Point(x, y) for x, y in zip(df['lon'], df['lat'])]
    crop_name = farm["common_crop_name"].iloc[0]
    return {'Area': area, 'Location': location, 'Crop': crop_name, 'Yield': crop_yield}


In [32]:
# test
farm_id = '0369f026-1f90-11ee-b788-0242ac150004'
get_farm_data(farm_id)

{'Area': 0.1409,
 'Location': [<POINT (-71.519 46.476)>],
 'Crop': 'Soybean',
 'Yield': 153158.2682753726}